In [1]:
# Set up folders
from EDA_functions import folders_set_up
import os

# Work with datarames
import pandas as pd
import numpy as np

# Evaluate models
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np

from IPython.display import Image

from surprise import Dataset
from surprise import Reader
from surprise import SVD # Matrix Factorization, Single Value Decomposition
from surprise.model_selection import cross_validate

#from scipy.sparse import spmatrixc

### Folders
Run the code below if you have the following structure:
- Group-project: GitHub folder
- 01 Input
- 02 Output

In [2]:
analysis_folder, input_folder, output_folder = folders_set_up.generate_folders()

In [3]:
# Import ratings data
title_level_df = pd.read_csv(os.path.join(input_folder,'Books_rating.csv'))

title_level_df.columns

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

In [4]:
# Import train indices
train_indices = pd.read_csv(
    os.path.join(output_folder, "train_indices.csv")
)

train_indices.columns

Index(['index_key', 'Title', 'index'], dtype='object')

In [5]:
# Keep ratings of books in the training dataset
recommender_df = pd.merge(
    train_indices,
    title_level_df,
    on = 'Title',
    how = 'left'
)

recommender_df.shape

(800337, 12)

In [6]:
recommender_df['Title'].nunique()

21419

In [7]:
recommender_df.columns

Index(['index_key', 'Title', 'index', 'Id', 'Price', 'User_id', 'profileName',
       'review/helpfulness', 'review/score', 'review/time', 'review/summary',
       'review/text'],
      dtype='object')

In [8]:
# Drop variables

# https://surprise.readthedocs.io/en/stable/reader.html
reader = Reader(rating_scale=(0.5, 4.0))
#reader = Reader()

data = Dataset.load_from_df(recommender_df[['Id', 'Title', 'review/score']], reader)


In [9]:
svd = SVD()

cross_validate(
    svd, 
    data, 
    measures=['RMSE', 'MAE', 'MSE'], # RMSE - Root Mean Square Error (MSE)
    cv=3, 
    verbose=True
)

Evaluating RMSE, MAE, MSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.1764  1.1746  1.1771  1.1760  0.0010  
MAE (testset)     0.9752  0.9740  0.9762  0.9751  0.0009  
MSE (testset)     1.3839  1.3796  1.3854  1.3830  0.0024  
Fit time          2.40    2.36    2.36    2.37    0.02    
Test time         0.99    0.98    0.98    0.99    0.00    


{'test_rmse': array([1.17637302, 1.17458453, 1.17705037]),
 'test_mae': array([0.97521409, 0.97398882, 0.97621272]),
 'test_mse': array([1.38385348, 1.37964881, 1.38544757]),
 'fit_time': (2.3990728855133057, 2.359369993209839, 2.362720012664795),
 'test_time': (0.9875919818878174, 0.9846389293670654, 0.9838318824768066)}